<a href="https://colab.research.google.com/github/akshaygrao77/Research-Work-Public-Second-Review/blob/main/Maximum_edges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from graphviz import Digraph,Source
import numpy as np
import math

In [ ]:
from IPython import display
from time import sleep
import copy
from pathlib import Path
import csv 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
class Vertex:
  def __init__(self,vertex_num):
    # List of vertices(objects) towards which edge go out of current vertex
    self.edges_out = []
    # List of vertices(objects) from which edge come towards current vertex
    self.edges_in = []
    # Layer number(starts from 0 as source) in which current vertex is present
    self.layer_num = -1
    # Vertex number. Acts as unique identifier for vertex
    self.vertex_num = vertex_num

  def __eq__(self, other) :
    return self.vertex_num == other.vertex_num
  
  def __str__(self):
    return "edges_out: "+str([v.vertex_num for v in self.edges_out]) + "\n layer_num: "+str(self.layer_num)+"\n vertex_num: "+str(self.vertex_num)

In [ ]:
class Layer:
  def __init__(self,list_of_vertices):
    # Holds all vertex objects in current layer
    self.list_of_vertices = list_of_vertices
    
  def __str__(self):
    return "list_of_vertices: "+str([v.vertex_num for v in self.list_of_vertices])

In [ ]:
class DAG:
  def __init__(self):
    # List of layer objects
    self.layers = []
    self.num_vertices = 0
    self.num_edges = 0
    self.vertex_num_vertex_obj_dict = dict()

  def __str__(self):
    layer_out = ""
    for each_layer in self.layers:
      layer_out += str(each_layer.list_of_vertices[0].layer_num)+"-->"+str([vertex.vertex_num for vertex in each_layer.list_of_vertices])+"\n"
    return "num_edges: "+str(self.num_edges) + "\n num_vertices: "+str(self.num_vertices)+"\n layers: "+str(layer_out)

In [ ]:
class RenderEpisode:
  def __init__(self,multi_staged_graph,title,list_of_removed_edges,edge_path_dict):
    self.multi_staged_graph = copy.deepcopy(multi_staged_graph) 
    self.title = title
    self.list_of_removed_edges = list_of_removed_edges.copy()
    self.edge_path_dict = copy.deepcopy(edge_path_dict) 


In [ ]:
def render_layered_digraphs(layered_dag,title,list_of_removed_edges = [],edge_path_dict={},dot=None,name="Graph",uniq_prefix=""):
  if(dot is None):
    dot = Digraph(name)
    dot.attr(label = title,labelloc = "t")
    # dot.body.append('label='+str(title))

  for each_layer in layered_dag.layers:
    for each_vertex in each_layer.list_of_vertices:
      dot.node(str(uniq_prefix)+str(each_vertex.vertex_num),str(each_vertex.vertex_num))

  for each_layer in layered_dag.layers:
    for each_vertex in each_layer.list_of_vertices:
      for each_dest_vertex in each_vertex.edges_out:
        if(not(edge_path_dict is None) and (each_vertex.vertex_num,each_dest_vertex.vertex_num) in edge_path_dict):
          dot.edge(str(uniq_prefix)+str(each_vertex.vertex_num), str(uniq_prefix)+str(each_dest_vertex.vertex_num),color="green",label=str(edge_path_dict[(each_vertex.vertex_num,each_dest_vertex.vertex_num)]))
        else:
          dot.edge(str(uniq_prefix)+str(each_vertex.vertex_num), str(uniq_prefix)+str(each_dest_vertex.vertex_num),color="green")  

  for (each_src,each_dest) in list_of_removed_edges:
    if(not(edge_path_dict is None) and (each_src.vertex_num,each_dest.vertex_num) in edge_path_dict):
      dot.edge(str(uniq_prefix)+str(each_src.vertex_num), str(uniq_prefix)+str(each_dest.vertex_num),color="red",label=str(edge_path_dict[(each_src.vertex_num,each_dest.vertex_num)]))  
    else:
      dot.edge(str(uniq_prefix)+str(each_src.vertex_num), str(uniq_prefix)+str(each_dest.vertex_num),color="red")
  
  return dot

In [ ]:
def render_list_of_episodes_as_subgraph(list_of_episodes):
  dot = Digraph()
  for each_episode_indx in range(len(list_of_episodes)):
    each_episode = list_of_episodes[each_episode_indx]
    current_render = render_layered_digraphs(each_episode.multi_staged_graph,each_episode.title,each_episode.list_of_removed_edges,each_episode.edge_path_dict,None,"cluster_"+str(each_episode_indx),str(each_episode_indx)+"_")
    dot.subgraph(current_render)
  return dot

In [ ]:
def generate_DAG_skeleton_based_on_layer_config(list_of_number_vertices_in_each_layer):
  return_dag = DAG()
  current_vertex_no = 0
  current_layer_num = 0
  layers = []
  for each_layer_no_of_vertex in list_of_number_vertices_in_each_layer:
    current_layer_list_of_vertices = []
    for each_vertex_in_current_layer in range(each_layer_no_of_vertex):
      current_vertex = Vertex(current_vertex_no)
      return_dag.vertex_num_vertex_obj_dict[current_vertex_no] = current_vertex
      current_vertex_no += 1
      current_vertex.layer_num = current_layer_num
      current_layer_list_of_vertices.append(current_vertex)
    
    current_layer_num += 1
    current_layer = Layer(current_layer_list_of_vertices)
    layers.append(current_layer)
  
  return_dag.layers = layers
  return_dag.num_vertices = current_vertex_no
  
  return return_dag

In [ ]:
def generate_DAG_based_on_layer_config(list_of_number_vertices_in_each_layer,mode="FULLY_TRANSITIVE"):
  return_dag = generate_DAG_skeleton_based_on_layer_config(list_of_number_vertices_in_each_layer)
  num_edges = 0

  if(mode == "FULLY_TRANSITIVE"):
    layers = return_dag.layers
    for source_layer_index in range(len(layers)):
      for dest_layer_index in range(source_layer_index+1,len(layers)):
        for src_vertex in layers[source_layer_index].list_of_vertices:
          for dest_vertex in layers[dest_layer_index].list_of_vertices:
            num_edges += 1
            src_vertex.edges_out.append(dest_vertex)
            dest_vertex.edges_in.append(src_vertex)
  elif(mode == "TRANSITIVE_AFTER_L0"):
    layers = return_dag.layers
    for source_layer_index in range(len(layers)):
      if(source_layer_index > 0):
        for dest_layer_index in range(source_layer_index+1,len(layers)):
          for src_vertex in layers[source_layer_index].list_of_vertices:
            for dest_vertex in layers[dest_layer_index].list_of_vertices:
              num_edges += 1
              src_vertex.edges_out.append(dest_vertex)
              dest_vertex.edges_in.append(src_vertex)
      else:
        dest_layer_index = source_layer_index+1
        for src_vertex in layers[source_layer_index].list_of_vertices:
            for dest_vertex in layers[dest_layer_index].list_of_vertices:
              num_edges += 1
              src_vertex.edges_out.append(dest_vertex)
              dest_vertex.edges_in.append(src_vertex)
    
    return_dag.num_edges = num_edges
  
  return return_dag
        

In [ ]:
def initialise_DP_nested_map(multi_stage_graph):
  list_of_vertex_num = []
  nested_dp_map = dict()
  for each_layer in multi_stage_graph.layers:
    for each_vertex in each_layer.list_of_vertices:
      list_of_vertex_num.append(each_vertex.vertex_num)
  
  for i in list_of_vertex_num:
    for j in list_of_vertex_num:
      val = 0
      if(i == j):
        val = 1
      if i in nested_dp_map:
        nested_dp_map[i][j] = val
      else:
        nested_dp_map[i]=dict()
        nested_dp_map[i][j] = val

  return nested_dp_map

In [ ]:
def number_of_paths_between_source_destination_in_multi_stage_graph(src_vertex_num,dest_vertex_num,multi_stage_graph,global_all_vertex_pair_DP):
  # print("src_vertex_num: "+str(src_vertex_num)+" dest_vertex_num:"+str(dest_vertex_num))
  src_vertex = multi_stage_graph.vertex_num_vertex_obj_dict[src_vertex_num]
  dest_vertex = multi_stage_graph.vertex_num_vertex_obj_dict[dest_vertex_num]
  
  if(global_all_vertex_pair_DP is None):
    global_all_vertex_pair_DP =initialise_DP_nested_map(multi_stage_graph)

  if(global_all_vertex_pair_DP[src_vertex.vertex_num][dest_vertex.vertex_num] != 0):
    return global_all_vertex_pair_DP[src_vertex.vertex_num][dest_vertex.vertex_num]

  dest_layer_num = dest_vertex.layer_num
  if(dest_layer_num == 0):
    return 0

  for layer_num in range(dest_layer_num-1, -1, -1) :
    list_of_vertices_in_previous_layer = multi_stage_graph.layers[layer_num].list_of_vertices
    for each_previous_layer_vertex in list_of_vertices_in_previous_layer:
      list_of_outgoing_edge_vertices = each_previous_layer_vertex.edges_out
      if(global_all_vertex_pair_DP[each_previous_layer_vertex.vertex_num][dest_vertex.vertex_num] == 0):
        for each_vertex in list_of_outgoing_edge_vertices:
          global_all_vertex_pair_DP[each_previous_layer_vertex.vertex_num][dest_vertex.vertex_num] += global_all_vertex_pair_DP[each_vertex.vertex_num][dest_vertex.vertex_num]
      
      if(each_previous_layer_vertex == src_vertex):
        # print("global_all_vertex_pair_DP:"+str(global_all_vertex_pair_DP))
        return global_all_vertex_pair_DP[each_previous_layer_vertex.vertex_num][dest_vertex.vertex_num]
  return 0

In [ ]:
def num_paths_in_multi_stage_graph_source_to_sink(multi_stage_graph):
  src_vertex_num = multi_stage_graph.layers[0].list_of_vertices[0].vertex_num
  dest_vertex_num = multi_stage_graph.layers[len(multi_stage_graph.layers) - 1].list_of_vertices[0].vertex_num

  return number_of_paths_between_source_destination_in_multi_stage_graph(src_vertex_num,dest_vertex_num,multi_stage_graph,None)

In [ ]:
def number_of_paths_passing_through_edge_in_multi_stage_graph(u,v,multi_stage_graph,global_all_vertex_pair_DP):
  src_vertex_num = multi_stage_graph.layers[0].list_of_vertices[0].vertex_num
  dest_vertex_num = multi_stage_graph.layers[len(multi_stage_graph.layers)-1].list_of_vertices[0].vertex_num
  
  number_of_paths_reaching_edge = number_of_paths_between_source_destination_in_multi_stage_graph(src_vertex_num,u,multi_stage_graph,global_all_vertex_pair_DP)
  number_of_paths_leaving_edge = number_of_paths_between_source_destination_in_multi_stage_graph(v,dest_vertex_num,multi_stage_graph,global_all_vertex_pair_DP)

  return number_of_paths_reaching_edge * number_of_paths_leaving_edge

In [ ]:
def remove_vertex_if_without_edges(vertexObj,multi_stage_graph):
  if(len(vertexObj.edges_out) == 0 and len(vertexObj.edges_in) == 0):
    # print("deleting: "+str(vertexObj))
    multi_stage_graph.layers[vertexObj.layer_num].list_of_vertices.remove(vertexObj)
    del multi_stage_graph.vertex_num_vertex_obj_dict[vertexObj.vertex_num]
    multi_stage_graph.num_vertices -= 1


In [ ]:
def eliminate_edge_involving_least_paths_from_multi_stage_graph(multi_stage_graph,global_all_vertex_pair_DP,preserve_vertex_count,elimination_start_layer_indx=0):
  least_path_edge_src = None
  least_path_edge_dest = None
  least_path_value = None
  edge_path_dict = dict()
  src_vertex = None
  dest_vertex = None

  for each_layer_indx in range(elimination_start_layer_indx,len(multi_stage_graph.layers)):
    each_layer = multi_stage_graph.layers[each_layer_indx]
    for each_vertex in each_layer.list_of_vertices:
      for each_edge_dest in each_vertex.edges_out:
        src_vertex_num = each_vertex.vertex_num
        dest_vertex_num =  each_edge_dest.vertex_num
        # print("uuuuuuuu src_vertex_num: "+str(src_vertex_num)+" dest_vertex_num:"+str(dest_vertex_num))
        if(preserve_vertex_count == False or (len(each_vertex.edges_out) > 1 and len(each_edge_dest.edges_in) > 1)):
          if(least_path_edge_src is None):
            least_path_edge_src = src_vertex_num
            least_path_edge_dest = dest_vertex_num
            least_path_value = number_of_paths_passing_through_edge_in_multi_stage_graph(src_vertex_num,dest_vertex_num,multi_stage_graph,global_all_vertex_pair_DP)
            edge_path_dict[(src_vertex_num,dest_vertex_num)] = least_path_value
          else:
            path_value = number_of_paths_passing_through_edge_in_multi_stage_graph(src_vertex_num,dest_vertex_num,multi_stage_graph,global_all_vertex_pair_DP)
            edge_path_dict[(src_vertex_num,dest_vertex_num)] = path_value
            if(path_value < least_path_value):
              least_path_edge_src = src_vertex_num
              least_path_edge_dest = dest_vertex_num
              least_path_value = path_value
  
  if(not(least_path_value is None) and not(least_path_edge_src is None) and not(least_path_edge_dest is None)):
    src_vertex = multi_stage_graph.vertex_num_vertex_obj_dict[least_path_edge_src]
    dest_vertex = multi_stage_graph.vertex_num_vertex_obj_dict[least_path_edge_dest]
  
  # Remove dest vertex from edge_out at src vertex and all other structures are updated accordingly
  src_vertex.edges_out.remove(dest_vertex)
  dest_vertex.edges_in.remove(src_vertex)
  multi_stage_graph.num_edges -= 1

  if(preserve_vertex_count == False):
    remove_vertex_if_without_edges(src_vertex,multi_stage_graph)
    remove_vertex_if_without_edges(dest_vertex,multi_stage_graph)

  return src_vertex,dest_vertex,edge_path_dict




In [ ]:
# display_mode= 'END_RESULT'=> Show end result in screen , 'WHL_EP_END"=> Show all sequence steps on screen at once at end , "ECH_EP" => Each elimination is shown with time gap of 3 seconds
def recurssively_eliminate_edges_in_DAG(layered_multi_stage_graph,target_num_of_edges,preserve_vertex_count=True,display_mode="END_RESULT",elimination_start_layer_indx=0):
  multi_stage_graph = copy.deepcopy(layered_multi_stage_graph)
  list_of_removed_edge_tuples = []
  list_of_render_episodes = []
  edge_path_dict = None
  global_all_vertex_pair_DP = initialise_DP_nested_map(multi_stage_graph)
  final_render = None
  title=""
  current_num_of_paths = 0

  current_episode = RenderEpisode(multi_stage_graph,"Before eliminations Paths:"+str(num_paths_in_multi_stage_graph_source_to_sink(multi_stage_graph))+" Edges:"+str(multi_stage_graph.num_edges),[],{})
  list_of_render_episodes.append(current_episode)
  
  current_num_of_edges = multi_stage_graph.num_edges
  
  while(current_num_of_edges > target_num_of_edges):
    src_vertex,dest_vertex,edge_path_dict = eliminate_edge_involving_least_paths_from_multi_stage_graph(multi_stage_graph,global_all_vertex_pair_DP,preserve_vertex_count,elimination_start_layer_indx)
    if(src_vertex is None or dest_vertex is None):
      break
    
    list_of_removed_edge_tuples.append((src_vertex,dest_vertex))
    current_num_of_edges = multi_stage_graph.num_edges
    global_all_vertex_pair_DP = initialise_DP_nested_map(multi_stage_graph)
    current_num_of_paths = num_paths_in_multi_stage_graph_source_to_sink(multi_stage_graph)

    # print("current_num_of_edges: "+str(current_num_of_edges)+"  current_num_of_paths:"+str(current_num_of_paths)+" Vertices:"+str(multi_stage_graph.num_vertices))
    title = "Paths:"+str(current_num_of_paths)+" Edges:"+str(current_num_of_edges)+" Vertices:"+str(multi_stage_graph.num_vertices)
    if(not(display_mode is None) and display_mode == "ECH_EP"):
      render = render_layered_digraphs(multi_stage_graph,title,list_of_removed_edge_tuples,edge_path_dict)
      display.display(render)
      display.clear_output(wait=True)
      sleep(3)
  
    current_episode = RenderEpisode(multi_stage_graph,title,list_of_removed_edge_tuples,edge_path_dict)
    list_of_render_episodes.append(current_episode)
  
  current_episode = RenderEpisode(multi_stage_graph,"After all eliminations "+str(title),[],{})
  list_of_render_episodes.append(current_episode)

  final_render = render_layered_digraphs(multi_stage_graph,title,list_of_removed_edge_tuples,edge_path_dict)
  if(not(display_mode is None) and (display_mode == "END_RESULT" or display_mode == "ECH_EP")):
    display.clear_output(wait=True)
    display.display(final_render)

  whole_sequence_render = render_list_of_episodes_as_subgraph(list_of_render_episodes)
  if(not(display_mode is None) and display_mode == "WHL_EP_END"):
    display.display(whole_sequence_render)

  return multi_stage_graph,list_of_removed_edge_tuples,edge_path_dict,whole_sequence_render,final_render

In [ ]:
list_of_layer_config=[
            [1,5,4,3,2,5,4,3,2,5,4,3,2,1],
            [1,5,4,3,4,5,4,3,4,5,4,3,4,1],
            [1,5,4,3,4,5,4,3,4,5,4,3,1],
            [1,5,4,3,2,5,4,3,2,5,4,3,1],
            [1,5,4,3,5,4,3,5,4,3,5,1],
            [1,5,4,3,2,5,4,3,2,4,3,1],
            [1,4,3,2,4,3,2,4,3,2,4,1],
            [1,5,4,3,2,5,4,3,2,4,5,1],
            [1,5,4,3,2,5,4,3,2,4,1,1],
            [1,5,4,3,2,5,4,3,2,4,1],
            [1,5,4,3,2,5,4,3,2,1,1],
            [1,5,4,3,2,5,4,3,2,1],
            [1,5,4,3,2,5,4,3,4,1],
            [1,4,4,4,4,4,4,4,2,1],
            [1,4,4,4,4,2,4,4,2,1],
            [1,6,6,6,6,6,6,6,6,1],
            [1,4,4,4,4,4,4,4,4,1],
            [1,3,4,3,4,3,4,3,1],
            [1,3,4,3,4,3,4,1],
            [1,5,4,5,4,5,4,1],
            [1,5,6,5,6,5,6,1] ,
            [1,5,6,5,6,5,1],
            [1,5,4,5,4,5,1],
            [1,3,4,3,4,3,1],
            [1,5,4,5,1] ,
            [1,5,4,1]
]

In [ ]:
def append_write_or_create_write_to_csv_file(csv_file_name,row,fields):
  my_file = Path(csv_file_name)
  if my_file.is_file():
    with open(csv_file_name, 'a') as csvfile:
      # Pass this file object to csv.writer()
      # and get a writer object
      writer_object = csv.writer(csvfile)
  
      # Pass the list as an argument into
      # the writerow()
      writer_object.writerow(row)
  
      #Close the file object
      csvfile.close()
  else:
    with open(csv_file_name, 'w') as csvfile: 
      # creating a csv writer object 
      csvwriter = csv.writer(csvfile) 
        
      # writing the fields 
      csvwriter.writerow(fields) 
        
      # writing the data rows 
      csvwriter.writerow(row)

      csvfile.close()

In [ ]:
def run_elimination_save_all_results(layer_config,target_edges,preserve_vertex_count=True,conf_name=None,display_mode=None,save_whole_sequence=False,generate_mode="FULLY_TRANSITIVE",elimination_start_layer_indx=0):
  layered_dag = generate_DAG_based_on_layer_config(layer_config,generate_mode)
  num_paths_before_elimination = num_paths_in_multi_stage_graph_source_to_sink(layered_dag)
  postfix_save_file = '_conf'
  conf_folder_name = 'root/outputs/target_edge_'+str(target_edges)
  postfix_folder_name = ""
  if(conf_name is None):
    for each_conf in layer_config:
      postfix_folder_name += '_'+str(each_conf)
  else:
    postfix_folder_name += conf_name
  
  postfix_save_file += postfix_folder_name
  postfix_save_file += '_ed_'+str(layered_dag.num_edges)+'_pt_'+str(num_paths_before_elimination)

  before_elimination_render = render_layered_digraphs(layered_dag,"Paths:"+str(num_paths_before_elimination)+" Edges:"+str(layered_dag.num_edges)+" Vertices:"+str(layered_dag.num_vertices))

  eliminated_graph,list_of_removed_edge_tuples,edge_path_dict,whole_sequence_render,final_render = recurssively_eliminate_edges_in_DAG(layered_dag,target_edges,preserve_vertex_count,display_mode,elimination_start_layer_indx)
  num_paths_after_elimination = num_paths_in_multi_stage_graph_source_to_sink(eliminated_graph)
  
  conf_folder_name += "/pt_aft_elim_"+str(num_paths_after_elimination)
  conf_folder_name += "/conf" + str(postfix_folder_name)

  before_elimination_render.render(filename = str(conf_folder_name)+'/before_elimination_render_'+str(postfix_save_file))
  final_render.render(filename = str(conf_folder_name)+'/after_elim_final_render_'+str(postfix_save_file))

  after_elimination_render = render_layered_digraphs(eliminated_graph,"Paths:"+str(num_paths_after_elimination)+" Edges:"+str(eliminated_graph.num_edges)+" Vertices:"+str(eliminated_graph.num_vertices))
  after_elimination_render.render(filename = str(conf_folder_name)+'/after_elimination_render_'+str(postfix_save_file))
  if(save_whole_sequence == True):
    whole_sequence_render.render(filename = str(conf_folder_name)+'/elimination_sequence_render_'+str(postfix_save_file))

  csv_file_name = 'root/outputs/target_edge_'+str(target_edges)+"/info_target_edge_"+str(target_edges)+".csv"
  row = [str(layer_config),num_paths_before_elimination,layered_dag.num_edges,layered_dag.num_vertices,num_paths_after_elimination,eliminated_graph.num_edges,eliminated_graph.num_vertices]
  fields = ['Config','Paths_before_elimination', 'Edges_before_elimination', 'Vertices_before_elimination', 'Paths_after_elimination','Edges_after_elimination','Vertices_after_elimination'] 
  
  append_write_or_create_write_to_csv_file(csv_file_name,row,fields)


  return num_paths_before_elimination,layered_dag.num_edges,layered_dag.num_vertices,num_paths_after_elimination,eliminated_graph.num_edges,eliminated_graph.num_vertices

In [ ]:
def run_elimination_over_partial_DNN_config(Din,d,k,preserve_vertex_count=True,conf_name=None,display_mode=None,save_whole_sequence=False):
  target_edges = (Din + 1)*d + (k-1)*d*d
  print("target_edges:"+str(target_edges))
  conf_name = "_DNN_Din"+str(Din)+"_d_"+str(d)+"_k_"+str(k)
  temp = [1] *(d*k+1)
  layer_config = [1,Din]
  layer_config+= (temp)
  run_elimination_save_all_results(layer_config,target_edges,preserve_vertex_count,conf_name,display_mode,generate_mode="TRANSITIVE_AFTER_L0",elimination_start_layer_indx=1)

In [ ]:
# run_elimination_over_partial_DNN_config(6,2,2,preserve_vertex_count=True,display_mode = "WHL_EP_END",save_whole_sequence=True)
run_elimination_over_partial_DNN_config(784,2,2,preserve_vertex_count=True)

target_edges:1574


In [ ]:
def run_elimination_save_all_results_on_different_configs(list_of_layer_config,target_edges,preserve_vertex_count,conf_name=None,display_mode=None,save_whole_sequence=False):
  for each_layer_config in list_of_layer_config:
    run_elimination_save_all_results(each_layer_config,target_edges,preserve_vertex_count,conf_name,display_mode,save_whole_sequence)
    print("Completed running layer_config:"+str(each_layer_config))

In [ ]:
# run_elimination_save_all_results_on_different_configs([[1,1,1,1,1,1,1,1,1,1]],11,True,None)

In [ ]:
def run_elimination_on_various_E_by_N_ratios_over_transitive_graphs(fixed_num_edges,list_of_EN_ratios=[1,2,3,4,5,6,7,8,9,10],display_mode=None,save_whole_sequence=False):
  preserved_case_return_val = []
  non_preserved_case_return_val = []
  standard_DNN_num_paths = []
  for each_EN_ratio in list_of_EN_ratios:
    current_num_vertex = math.ceil(fixed_num_edges/each_EN_ratio)
    current_config = [1] * current_num_vertex
    
    conf_name = "_nopres_trnsitive_"+str(current_num_vertex)
    no_pres_pt_bef_el,no_pres_ed_bef_el,no_pres_v_bef_el,no_pres_pt_aft_el,no_pres_ed_aft_el,no_pres_v_aft_el = run_elimination_save_all_results(current_config,fixed_num_edges,False,conf_name,display_mode,save_whole_sequence)
    non_preserved_case_return_val.append([no_pres_pt_bef_el,no_pres_ed_bef_el,no_pres_v_bef_el,no_pres_pt_aft_el,no_pres_ed_aft_el,no_pres_v_aft_el])
    
    conf_name = "_pres_trnsitive_"+str(current_num_vertex)
    if(no_pres_v_bef_el == no_pres_v_aft_el):
      preserved_case_return_val.append([no_pres_pt_bef_el,no_pres_ed_bef_el,no_pres_v_bef_el,no_pres_pt_aft_el,no_pres_ed_aft_el,no_pres_v_aft_el])
    else:
      pres_pt_bef_el,pres_ed_bef_el,pres_v_bef_el,pres_pt_aft_el,pres_ed_aft_el,pres_v_aft_el = run_elimination_save_all_results(current_config,fixed_num_edges,True,conf_name,display_mode,save_whole_sequence)
      preserved_case_return_val.append([pres_pt_bef_el,pres_ed_bef_el,pres_v_bef_el,pres_pt_aft_el,pres_ed_aft_el,pres_v_aft_el])
    
    
    current_num_paths_in_std_dnn = math.pow(each_EN_ratio,math.ceil((current_num_vertex - 2)/each_EN_ratio))
    standard_DNN_num_paths.append(current_num_paths_in_std_dnn)

    print("Completed running layer_config:"+str(current_config)+" config:"+str(conf_name))
  
  # diamond_shape_num_paths = math.pow(3,math.ceil(fixed_num_edges/3))
  
  return list_of_EN_ratios,preserved_case_return_val,non_preserved_case_return_val,standard_DNN_num_paths

In [ ]:
# list_of_EN_ratios,preserved_case_return_val,non_preserved_case_return_val,standard_DNN_num_paths,diamond_shape_num_paths = run_elimination_on_various_E_by_N_ratios_over_transitive_graphs(30)

In [ ]:
def generate_point_label_and_paths(list_of_values):
  list_data_point_label = []
  list_paths = []
  for each_sublist in list_of_values:
    current_label = "BP:"+str(each_sublist[0])+"_BE:"+str(each_sublist[1])+"_BV:"+str(each_sublist[2])+"_AP"+str(each_sublist[3])+"_AE"+str(each_sublist[4])+"_AV"+str(each_sublist[5])
    list_data_point_label.append(current_label)
    list_paths.append(each_sublist[3])
  
  return list_data_point_label,list_paths

In [ ]:
def plot_path_comparision_over_EN_ratios(list_of_EN_ratios,list_preserved_val,list_non_preserved_val,list_standard_DNN_num_paths,target_num_of_edges):
  fig = plt.figure(figsize=(20,18))
  # fig.set_size_inches(18.5, 10.5)
  
  x = list_of_EN_ratios
  list_data_pt_pres,y_pres = generate_point_label_and_paths(list_preserved_val)
  print("Paths using preserved setting"+str(y_pres))
  
  list_data_pt_non_pres,y_non_pres = generate_point_label_and_paths(list_non_preserved_val)
  print("Number of paths in non-preserved setting"+str(y_non_pres))
  
  y_std_dnn = list_standard_DNN_num_paths

  # y_diamond = [diamond_shape_num_paths] * len(list_of_EN_ratios)

  plt.plot(x, y_pres, label = "Preserving ratio")
  
  for x_indx,each_x in  enumerate(x):
    y = y_pres[x_indx]
    data_point_label = list_data_pt_pres[x_indx]
    plt.annotate('(%s, %s)' % (each_x,data_point_label), xy=(each_x,y), textcoords='data')

  plt.plot(x, y_non_pres, label = "Not preserving vertex ratio")

  # plt.plot(x, y_diamond, label = "Diamond shaped case")

  plt.plot(x, y_std_dnn, label = "Standard DNN")
  
  plt.xticks(list_of_EN_ratios)

  plt.xlabel('E/N ratios')
  # naming the y axis
  plt.ylabel('Number of paths')
  # giving a title to my graph
  plt.title('Number of paths vs E/N ratios for target-edge:'+str(target_num_of_edges))
  plt.grid()

  # show a legend on the plot
  plt.legend()
  
  plt.savefig('root/plots/target_edge_'+str(target_num_of_edges)+'.png')
  plt.savefig('root/plots/target_edge_'+str(target_num_of_edges)+'.pdf')

  display.display(plt)
  sleep(3)
  display.clear_output(wait=True)
  # plt.show()
  return plt


In [ ]:
def plot_path_comparision_over_EN_ratios_for_different_configs(list_of_target_edges,list_of_EN_ratios=[1,2,3,4,5,6,7,8,9,10],display_mode=None,save_whole_sequence=False):
  for each_target_edge in list_of_target_edges:
    print("++++++++++++++++++++++++++++++++++++++++++++++++++++ Beginning to compute paths for target_edge:"+str(each_target_edge))
    list_of_EN_ratios,preserved_case_return_val,non_preserved_case_return_val,standard_DNN_num_paths = run_elimination_on_various_E_by_N_ratios_over_transitive_graphs(each_target_edge,list_of_EN_ratios,display_mode,save_whole_sequence)
    plot_path_comparision_over_EN_ratios(list_of_EN_ratios,preserved_case_return_val,non_preserved_case_return_val,standard_DNN_num_paths,each_target_edge)
    print("==================================================== Completed to compute paths for target_edge:"+str(each_target_edge))


In [ ]:
plot_path_comparision_over_EN_ratios_for_different_configs([48,51,54,57,60,63,66,69,72,75,78,81,84],[1.5,2,2.5,3,4,5,6,7,8,9,10])

In [ ]:
!zip -r /content/experiments-Din_k_d.zip /content/root

  adding: content/root/ (stored 0%)
  adding: content/root/outputs/ (stored 0%)
  adding: content/root/outputs/target_edge_1574/ (stored 0%)
  adding: content/root/outputs/target_edge_1574/info_target_edge_1574.csv (deflated 50%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/ (stored 0%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/conf_DNN_Din784_d_2_k_2/ (stored 0%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/conf_DNN_Din784_d_2_k_2/after_elim_final_render__conf_DNN_Din784_d_2_k_2_ed_4714_pt_12544.pdf (deflated 0%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/conf_DNN_Din784_d_2_k_2/after_elim_final_render__conf_DNN_Din784_d_2_k_2_ed_4714_pt_12544 (deflated 87%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/conf_DNN_Din784_d_2_k_2/before_elimination_render__conf_DNN_Din784_d_2_k_2_ed_4714_pt_12544.pdf (deflated 1%)
  adding: content/root/outputs/target_edge_1574/pt_aft_elim_3136/conf_DN